# Práctica Integrada: Pipeline de Preprocesamiento para Análisis de Feedback en E-Commerce

Esta práctica busca crear un pipeline robusto que preserve formatos útiles y limpie lo irrelevante.

## Fase 0: Importacion de librerias

In [1]:
%pip install emoji
%pip install dateparser

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ------------------------------------- 295.7/295.7 kB 19.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import re
import emoji
import dateparser


## Fase 1: Limpieza Contextual de Texto

In [3]:
# Cargar datos
with open("validation_samples.json", "r", encoding="utf-8") as f:
    samples = json.load(f)
print(samples[0])


{'input': '🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023.'}


Texto de muestra a limpiar

`🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023.`

In [4]:

def limpiar_texto(texto):
    protegidos = []

    def proteger(match):
        protegidos.append(match.group(0))
        return f"__PROT{len(protegidos)-1}__"

    # Regex para detectar $99.99, 25%, 3/4, etc.
    texto = re.sub(r"\$?\d+(?:[.,]\d+)?%?|\d+/\d+", proteger, texto)

    # Eliminar URLs y handles
    texto = re.sub(r"https?://\S+|www\.\S+|@\w+", "", texto)
    
    # Eliminar signos de puntuación no deseados (excepto ! ? % $ /)
    texto = re.sub(r"[\.,;:()\[\]\"'\\]", "", texto)
    
    # Unificar espacios
    texto = re.sub(r"\s+", " ", texto).strip()

    for i, val in enumerate(protegidos):
        texto = texto.replace(f"__PROT{i}__", val)
    
    return texto


In [5]:

print(limpiar_texto(samples[0]["input"]))


🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 antes $150 👟 ¡Válido hasta el 30/11/2023! Visita Atención ¿Envío gratis? 😃 #ModaDeportiva2023


## 🔄 Fase 2: Normalización de Números y Unidades

In [ ]:
#TODO
def normalizar_numeros(texto):
    # Normalizar fechas
    fechas = re.findall(r"\b\d{2}/\d{2}/\d{4}\b", texto)
    for fecha in fechas:
        fecha_iso = dateparser.parse(fecha).strftime("%Y-%m-%d")
        texto = texto.replace(fecha, fecha_iso)

    # Monedas
    texto = re.sub(r"\$\s?(\d+(\.\d{1,2})?)", r"<USD>\1", texto)
    texto = re.sub(r"(\d+(\.\d{1,2})?)\$", r"<USD>\1", texto)
    texto = re.sub(r"(\d+(\.\d{1,2})?)€", r"<EUR>\1", texto)
    
    # Unidades
    texto = re.sub(r"\b(\d+)(x|kg|ml)\b", r"\1_\2", texto)
    
    # Reemplazar números genéricos que no sean parte de lo anterior
    # texto = re.sub(r"(?<!<USD>|<EUR>|\d_)(?<!\d/)\b\d{2,}\b(?!%)", "<NUM>", texto)
    texto = re.sub(r"\b\d{2,}\b", lambda m: "<NUM>" if not re.search(r"<USD>|<EUR>|\d_|/", texto[m.start()-5:m.end()+1]) else m.group(0), texto)


    return texto


In [ ]:
#TODO
texto = limpiar_texto(samples[0]["input"])
print(normalizar_numeros(texto))


🔥¡OFERTA! Compre 2_x zapatos Nike a <USD>99.<NUM> antes <USD>150 👟 ¡Válido hasta el <NUM>-<NUM>-<NUM>! Visita Atención ¿Envío gratis? 😃 #ModaDeportiva2023


## 🔤 Fase 3: Normalización de Mayúsculas con Reconocimiento de Entidades

In [ ]:

import spacy
nlp = spacy.load("es_core_news_lg")

# Diccionario personalizado de marcas (simulado)
marcas_manual = {"Zara", "Nike", "Adidas", "iPhone"}

def normalizar_mayusculas(texto):
    doc = nlp(texto)
    resultado = []

    for token in doc:
        if token.text in marcas_manual:
            resultado.append(token.text)
        elif token.ent_type_ in {"ORG", "LOC", "PRODUCT", "GPE"}:
            resultado.append(token.text)
        elif token.text.startswith("#"):
            resultado.append(token.text)
        else:
            resultado.append(token.text.lower())
    
    return " ".join(resultado)


In [ ]:

texto_final = normalizar_mayusculas(normalizar_numeros(limpiar_texto(texto_prueba)))
print(texto_final)


## ✅ Verificación Final


El resultado esperado debe conservar formatos y significado semántico útil para análisis de feedback comercial.  
Ejemplo de salida:

```
🔥 oferta compre 2_unidades zapatos Nike a <USD>99.99 antes <NUM> 👟
válido hasta el 2023-11-30 atención ¿envío gratis 😃 #ModaDeportiva2023
```
